### Iterative testing after laoding model 

In [1]:
import mujoco
import numpy as np
import mujoco.viewer as mv

# Load URDF directly - MuJoCo handles the conversion internally
model = mujoco.MjModel.from_xml_path("/home/blank/Projects/Internship/sentienc/Robotic-Simulation/mujoco-experiments/models/S1/urdf/mjmodel.xml")
print("✅ Model loaded from URDF!")
data = mujoco.MjData(model)

✅ Model loaded from URDF!


In [2]:
# Reset state
mujoco.mj_resetData(model, data)

# Raise root HIGH
data.qpos[2] = 1.3   # start higher than needed
mujoco.mj_forward(model, data)

print("root z:", data.qpos[2])


root z: 1.3


In [3]:
stand_ctrl = np.zeros(model.nu)

# freeze upper body
stand_ctrl[0:14] = 0.0

# right leg
stand_ctrl[14] = 0.0
stand_ctrl[15] = -0.3
stand_ctrl[16] = data.qpos[16]   # hip yaw: HOLD current
stand_ctrl[17] = 0.6
stand_ctrl[18] = -0.3

# left leg
stand_ctrl[19] = 0.0
stand_ctrl[20] = -0.3
stand_ctrl[21] = data.qpos[21]
stand_ctrl[22] = 0.6
stand_ctrl[23] = -0.3


In [4]:
for step in range(2000):
    data.ctrl[:] = stand_ctrl
    mujoco.mj_step(model, data)

    if not np.isfinite(data.qacc).all():
        print("❌ NaN detected at step", step)
        break

    if step % 200 == 0:
        print(f"step {step}, root z = {data.qpos[2]:.3f}")


step 0, root z = 1.300
step 200, root z = -0.653
step 400, root z = -0.399
step 600, root z = 1.196
step 800, root z = -0.848
step 1000, root z = -0.280
step 1200, root z = 0.963
step 1400, root z = -0.598
step 1600, root z = -1.746
step 1800, root z = -4.678


In [5]:
model.opt.gravity[:] = 0


In [7]:
# only test right hip pitch
stand_ctrl[:] = data.qpos[:model.nu]
stand_ctrl[15] = data.qpos[15] - 0.2   # r hip pitch



In [8]:
for i in range(model.nu):
    print(i,
          mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_ACTUATOR, i),
          model.actuator_gainprm[i][0])


0 act_torso 600.0
1 act_waist 400.0
2 act_l_shoulder_pitch 0.0
3 act_l_shoulder_roll 0.0
4 act_l_elbow_yaw 0.0
5 act_l_elbow_forearm 0.0
6 act_l_wrist_yaw 0.0
7 act_r_shoulder_pitch 0.0
8 act_r_shoulder_roll 0.0
9 act_r_elbow_yaw 0.0
10 act_r_elbow_forearm 0.0
11 act_r_wrist_yaw 0.0
12 act_neck_yaw 0.0
13 act_head 0.0
14 act_r_hip_roll 0.0
15 act_r_hip_pitch 400.0
16 act_r_hip_yaw 0.0
17 act_r_knee 500.0
18 act_r_foot 300.0
19 act_l_hip_roll 0.0
20 act_l_hip_pitch 400.0
21 act_l_hip_yaw 0.0
22 act_l_knee 500.0
23 act_l_foot 300.0


In [9]:
mujoco.mj_resetData(model, data)

# reset free joint to identity
data.qpos[0:4] = [1, 0, 0, 0]   # unit quaternion
data.qpos[4:7] = [0, 0, 1.5]   # position well above ground

data.qvel[:] = 0
mujoco.mj_forward(model, data)


In [10]:
print(data.qpos[0:4])

[1. 0. 0. 0.]


In [ ]:
with mv.launch_passive(model, data) as viewer:
    while viewer.is_running():
        mujoco.mj_forward(model, data)   # 👈 THIS IS THE KEY
        viewer.sync()


Failed to load plugin 'libdecor-gtk.so': failed to init
/home/blank/Projects/Internship/sentienc/Robotic-Simulation/mujoco-experiments/.venv/lib64/python3.13/site-packages/glfw/__init__.py:917: GLFWError: (65548) b'Wayland: The platform does not provide the window position'
  warnings.warn(message, GLFWError)


: 